In [2]:
import tensorflow as tf
import numpy as np
!pip install pandas
import pandas as pd

     |████████████████████████████████| 9.5 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 1.0 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
lesions = {0: "Basal Cell Carcinoma", 
    1: "Lentigo",
    2: "Malignant Melanoma", 
    3: "Melanocytic naevus",
    4: "seborrhoeic keratosis",
    5: "Wart", 
    6: "Actinic Keratosis",
    7: "Squamous Cell Carcinoma",
    8: "Intraepithelial Carcinoma", 
    9: "Pyogenic Granuloma",
    10: "Haemangioma",
    11: "Dermatofibroma"
}

In [4]:
train = pd.read_csv('train.csv',index_col=0)
test = pd.read_csv('test.csv',index_col=0)
val = pd.read_csv('val.csv',index_col=0)

train_y = train['label'].values
test_y = test['label'].values
val_y = val['label'].values

train.head()

,id,label
0,dataset-split/train/Wart/Wart_original_100.jpg...,5
1,dataset-split/train/Wart/Wart_original_37.jpg_...,5
2,dataset-split/train/Wart/Wart_original_2.jpg_4...,5
3,dataset-split/train/Wart/Wart_original_21.jpg_...,5
4,dataset-split/train/Wart/Wart_original_54.jpg_...,5


In [5]:
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)
df_val = pd.DataFrame(val)

In [6]:
# step 1
train_dataset_filenames = df_train['id'].values.tolist()
train_dataset_labels = df_train['label'].values.tolist()
filenames = tf.constant(train_dataset_filenames)
labels = tf.constant(train_dataset_labels)

# step 2: create a dataset returning slices of `filenames`
train_dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

train_dataset = train_dataset.map(_parse_function)
train_dataset = train_dataset.batch(32)

# step 4: create iterator and final input tensor
iterator_train = tf.compat.v1.data.make_one_shot_iterator(train_dataset)
images_train, labels_train = iterator_train.get_next()

In [7]:
# step 1
val_dataset_filenames = df_val['id'].values.tolist()
val_dataset_labels = df_val['label'].values.tolist()
filenames = tf.constant(val_dataset_filenames)
labels = tf.constant(val_dataset_labels)

# step 2: create a dataset returning slices of `filenames`
val_dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

val_dataset = val_dataset.map(_parse_function)
val_dataset = val_dataset.batch(32)

# step 4: create iterator and final input tensor
iterator_val = tf.compat.v1.data.make_one_shot_iterator(val_dataset)
images_val, labels_val = iterator_val.get_next()

In [8]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

Num GPUs Available:  1
